## Table of Contents

1. [Where Are We Now?](#where-are-we-now)
2. [How Much Is 1 Degree?](#how-much-is-1-degree)
3. [What Does 1 Degree Mean Where I Live?](#what-does-1-degree-mean-where-i-live)
4. [What Are the Major Greenhouse Gases?](#what-are-the-major-greenhouse-gases)
5. [How Much Does Each Gas Contribute to Global Warming?](#how-much-does-each-gas-contribute-to-global-warming)
6. [What Are Our Greenhouse Gas Levels?](#what-are-our-greenhouse-gas-levels)
7. [How Much Carbon Is This?](#how-much-carbon-is-this)
8. [How Much Carbon Is in the Permafrost?](#how-much-carbon-is-in-the-permafrost)

## Where Are We Now?

In this section, we'll explore the current state of global temperatures. We'll break it into three parts:

1. [Downloading the Data](#downloading-the-data)
2. [Analyzing the Data](#analyzing-the-data)
3. [Showing the Data](#showing-the-data)

### Downloading the Data

I've downloaded the climate data for a few cities, but feel free to redownload them yourselves. I didn't find any APIs I liked, so I used data from [WeatherStats](https://www.weatherstats.ca) and global temperature data from the [University of Maine's Climate Reanalyzer](https://climatereanalyzer.org/).

In [ ]:
# Deterministic color mapping for each city and global average
color_mapping = {
    "Calgary": "#8c564b",          # Brown
    "Edmonton": "#e377c2",         # Pink
    "Global Average": "#9467bd",   # Purple
    "Halifax": "#7f7f7f",          # Gray
    "Montreal": "#bcbd22",         # Yellow-green
    "Ottawa": "#17becf",           # Cyan
    "Thunder Bay": "#ff1493",      # Deep pink
    "Toronto": "#ff7f0e",          # Orange
    "Sault Ste. Marie": "#1f77b4", # Blue
    "Vancouver": "#aec7e8",        # Light blue
    "Whitehorse": "#2ca02c",       # Green
    "Yellowknife": "#d62728"       # Red
}

# Example usage for cities:
city_files = {
    'Calgary': '/data/weatherstats_calgary_daily.csv',
    'Edmonton': '/data/weatherstats_edmonton_daily.csv',
    'Halifax': '/data/weatherstats_halifaxairport_daily.csv',
    'Montreal': '/data/weatherstats_montreal_daily.csv',
    'Ottawa': '/data/weatherstats_ottawa_daily.csv',
    'Sault Ste. Marie': '/data/weatherstats_saultstemarie_daily.csv',
    'Thunder Bay': '/data/weatherstats_thunderbay_daily.csv',
    'Toronto': '/data/weatherstats_toronto_daily.csv',
    'Vancouver': '/data/weatherstats_vancouver_daily.csv',
    'Whitehorse': '/data/weatherstats_whitehorse_daily.csv',
    'Yellowknife': '/data/weatherstats_yellowknife_daily.csv'
}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json

def process_daily_city_data(daily_city_file):
    """
    Process daily temperature data for a daily city file.

    Args:
        daily_file (str): filename with daily temperature data in CSV format.

    Returns:
        DataFrame: Processed daily data with 'date' and 'temperature' columns.
    """
    # Read the daily data
    daily_data = pd.read_csv(daily_city_file, low_memory=False)
    
    # Convert 'date' to datetime and set as index
    daily_data['date'] = pd.to_datetime(daily_data['date'])
    daily_data.set_index('date', inplace=True)
    daily_data['temperature'] = daily_data['avg_temperature']
    
    return daily_data[['date', 'temperature']]

def process_rolling_temperatures(daily_data):
    """
    Calculate the rolling average temperature over a 365-day window.

    Args:
        daily_data (DataFrame): DataFrame with 'date' and 'temperature' columns.

    Returns:
        DataFrame: DataFrame with 'date' and 'rolling_average_temp' columns.
    """
    result = daily_data.copy()
    result['rolling_average_temp_365'] = daily_data['temperature'].rolling(window=365).mean()
    
    return result

def process_global_temperature_json(json_file_path):
    """
    Process global temperature data from a JSON file to calculate daily averages and rolling averages.

    Args:
        json_file_path (str): Path to the JSON file containing global temperature data.

    Returns:
        DataFrame: Processed global temperature data with 'date' and 'rolling_average_temp' columns.
    """
    # Load global temperature data from JSON
    with open(json_file_path, 'r') as file:
        global_temperature_data = json.load(file)
    
    # Transform the JSON into a DataFrame
    rows = []
    for year_data in global_temperature_data:
        year = year_data["name"]
        daily_temps = year_data["data"]
        for day, temp in enumerate(daily_temps, start=1):
            rows.append({"Year": year, "Day": day, "Temperature": temp})
    
    global_temperature_df = pd.DataFrame(rows)
    
    # Clean the data (filter nulls, handle non-leap years)
    global_temperature_df = global_temperature_df[global_temperature_df['Temperature'].notnull()]
    global_temperature_df = global_temperature_df[~global_temperature_df['Year'].str.contains('-')]
    
    # Convert 'Year' and 'Day' to 'date' and calculate rolling average
    global_temperature_df['date'] = (pd.to_datetime(global_temperature_df['Year'], format='%Y') + 
                                      pd.to_timedelta(global_temperature_df['Day'] - 1, unit='D')).dt.date
    global_daily = global_temperature_df[['date', 'Temperature']].rename(columns={'Temperature': 'temperature'})
    
    global_daily['rolling_average_temp_365'] = global_daily['temperature'].rolling(window=365, min_periods=365).mean()
    global_daily = global_daily[['date', 'temperature', 'rolling_average_temp_365']]
    
    return global_daily

def plot_rolling_averages(dataframe_dict):
    """
    Plots the 365-day rolling average temperatures for multiple cities and global average.
    
    Args:
        dataframe_dict (dict): A dictionary where the keys are city names (strings) 
                               and the values are DataFrames containing 'date' and 'rolling_average_temp'.
    """
    plt.figure(figsize=(12, 6))
    plt.title("365-Day Rolling Average Temperatures", fontsize=16)
    
    # Loop through each city and plot the data
    for city, data in dataframe_dict.items():
        trimmed_data = data.dropna(subset=['rolling_average_temp_365'])
        color = color_mapping.get(city, "#000000")  # Default to black if city not in color_mapping
        plt.plot(trimmed_data['date'], trimmed_data['rolling_average_temp_365'], label=city, color=color, alpha=0.7)
    
    # Add labels and grid
    plt.xlabel("Date", fontsize=14)
    plt.ylabel("365-Day Rolling Average Temperature (°C)", fontsize=14)
    plt.xticks(rotation=45)  # Rotate date labels for better readability
    plt.legend()
    plt.grid(True)
    plt.tight_layout()  # Ensures everything fits without overlapping
    plt.show()

def plot_temperatures(dataframe_dict):
    """
    Plots the 365-day rolling average temperatures for multiple cities and global average.
    
    Args:
        dataframe_dict (dict): A dictionary where the keys are city names (strings) 
                               and the values are DataFrames containing 'date' and 'rolling_average_temp'.
    """
    plt.figure(figsize=(12, 6))
    plt.title("Daily Temperatures", fontsize=16)
    
    # Loop through each city and plot the data
    for city, data in dataframe_dict.items():
        color = color_mapping.get(city, "#000000")  # Default to black if city not in color_mapping
        plt.plot(data['date'], data['temperature'], label=city, color=color, alpha=0.7)
    
    # Add labels and grid
    plt.xlabel("Date", fontsize=14)
    plt.ylabel("365-Day Rolling Average Temperature (°C)", fontsize=14)
    plt.xticks(rotation=45)  # Rotate date labels for better readability
    plt.legend()
    plt.grid(True)
    plt.tight_layout()  # Ensures everything fits without overlapping
    plt.show()

def plot_temperatures_and_rolling_averages(dataframe_dict):
    """
    Plots the 365-day rolling average temperatures for multiple cities and global average.
    
    Args:
        dataframe_dict (dict): A dictionary where the keys are city names (strings) 
                               and the values are DataFrames containing 'date' and 'rolling_average_temp'.
    """
    plt.figure(figsize=(12, 6))
    plt.title("365-Day Rolling Average and Daily Temperatures", fontsize=16)
    
    # Loop through each city and plot the data
    for city, data in dataframe_dict.items():
        trimmed_data = data.dropna(subset=['rolling_average_temp_365'])
        color = color_mapping.get(city, "#000000")  # Default to black if city not in color_mapping
        plt.plot(data['date'], data['temperature'], label=city, color=color, linestyles="dashed", alpha=0.7)
        plt.plot(trimmed_data['date'], trimmed_data['rolling_average_temp_365'], label=city, color=color, linestyles="dotted", alpha=0.7)
    
    # Add labels and grid
    plt.xlabel("Date", fontsize=14)
    plt.ylabel("365-Day Rolling Average Temperature (°C)", fontsize=14)
    plt.xticks(rotation=45)  # Rotate date labels for better readability
    plt.legend()
    plt.grid(True)
    plt.tight_layout()  # Ensures everything fits without overlapping
    plt.show()

city_data = {}
city_hourly_data = {}
for city, file in city_files.items():
    hourly_data, daily_data = process_hourly_data(file)
    city_hourly_data[city] = hourly_data
    city_data[city] = process_rolling_daily(daily_data)

# Example usage for global temperature data:
city_data['Global Average'] = process_global_temperature_json('/kaggle/input/global-average-air-temperature/era5_world_t2_day.json')

# Call the plotting function
plot_rolling_averages(city_data)

### Analyzing the Data

Once we have the data, we'll process it to calculate rolling averages and other meaningful metrics.

### Showing the Data

Finally, we'll visualize the data to understand the trends and patterns in global temperatures.

## How Much Is 1 Degree?

## What Does 1 Degree Mean Where I Live?

## What Are the Major Greenhouse Gases?

## How Much Does Each Gas Contribute to Global Warming?

## What Are Our Greenhouse Gas Levels?

## How Much Carbon Is This?

## How Much Carbon Is in the Permafrost?